# Parallelized version

In [1]:
import math
import random
import numpy as np
np.random.seed(33)
global b  
b = 0

## Initializing spark

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setAppName("appName").setMaster("local")
sc = SparkContext()

22/11/23 13:59:59 WARN Utils: Your hostname, Alexs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.244 instead (on interface en0)
22/11/23 13:59:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/23 14:00:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Functions

### Auxiliar functions

In [4]:
def trainTestSplit(dataset):
    train = dataset.filter(lambda x: x[2]==1)
    train = train.map(lambda x: (x[0], x[1]))
    test = dataset.filter(lambda x: x[2]==0)
    test = test.map(lambda x: (x[0], x[1]))
    return (train, test)

In [5]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [6]:
def rdd_cost_function(RDD_Xyyhat, lambda_ref, m, w):
    y = RDD_Xyyhat[1]
    y_hat = RDD_Xyyhat[2]
    temp = (-1/m) * (
        y * np.log(y_hat) + (1 - y)
        * np.log(1 - y_hat)
    )
    cost = temp + (lambda_ref/(2*m))*sum([i*i for i in w.value])
    return cost

In [7]:
def str_to_number_list(line):
    numbers = line.replace(" ","").split(",")
    numbers = [float(number) for number in numbers]
    
    # Separate X and y
    numbers = (numbers[:-1], numbers[-1])
    
    return numbers

In [8]:
def sum_list_values(list1, list2):
    return [n1 + n2 for n1, n2 in zip(list1, list2)]

In [9]:
def sum_minus_mean_squared(numbers):
    return [(number-means.value[x])**2 for x,number in enumerate(numbers)]

In [10]:
def rdd_mean_by_column(RDD_Xy, m):
    means = RDD_Xy.map(lambda x: x[0]).reduce(sum_list_values)
    means = [x/m for x in means]
    return means

In [11]:
def rdd_mean_by_column(RDD_Xy, m):
    means = RDD_Xy.map(lambda x: x[0]).reduce(sum_list_values)
    means = [x/m for x in means]
    return means

In [12]:
def rdd_std_by_column(RDD_Xy, m):
    stds = RDD_Xy.map(lambda x: x[0]).map(sum_minus_mean_squared).reduce(sum_list_values)
    stds = [math.sqrt(x/m) for x in stds]
    return stds

In [13]:
def multiply_RDDXy_by_w(Xy):
    tot = 0
    for xi, wi in zip(Xy[0], w.value):
        tot += xi * wi
    return (Xy[0], Xy[1], sigmoid(tot))

In [14]:
def calculate_dw(RDD_Xyyhat):
    dw = []
    for x in RDD_Xyyhat[0]:
        dw.append((RDD_Xyyhat[1]-RDD_Xyyhat[2])*x)
    return dw

### Mandatory functions

In [15]:
def readFile(filename):
    dataset = sc.textFile(filename)
    dataset = dataset.map(str_to_number_list)
    return dataset.sample(False, 0.01)

In [16]:
def normalize(RDD_Xy):
    m = RDD_Xy.count()
    global means
    means = sc.broadcast(rdd_mean_by_column(RDD_Xy, m))
    stds = sc.broadcast(rdd_std_by_column(RDD_Xy, m))
    norm_rdd = RDD_Xy.map(lambda values: ([(x - mean) / std for x, mean, std in zip(values[0], means.value, stds.value)], values[1]))
    return norm_rdd

In [17]:
def train(RDD_Xy, iterations, learning_rate, lambda_reg):
    global b
    global m
    m = RDD_Xy.count()
    n = len(RDD_Xy.take(1)[0])
    
    global w
    w = sc.broadcast(np.random.rand(n))
    
    
    for it in range(iterations):
        RDD_Xyyhat = RDD_Xy.map(multiply_RDDXy_by_w)
        print(f"Cost for it {it}:", RDD_Xyyhat.map(lambda x: rdd_cost_function(x, lambda_reg, m, w)).reduce(lambda x,y: x+y))
        print(f"Accuracy for it {it}:{accuracy(w, b, RDD_Xy)}")
        dw = RDD_Xyyhat.map(calculate_dw).reduce(lambda x, y: [xi+yi for xi, yi in zip(x, y)])        
        dw_final = []
        for dwi, wi in zip(dw, w.value):
            dw_final.append( ((dwi+(lambda_reg*wi))/m) )
        w = sc.broadcast([wi - (learning_rate * dwi) for wi, dwi in zip(w.value, dw_final)])
        
        b -= learning_rate * (1/m)*RDD_Xyyhat.map(lambda x: x[1]-x[2]).sum()
    
    return w, b


In [18]:
def accuracy(ws, b, RDD_Xy):
    total = RDD_Xy.count()
    y_and_y_hat = RDD_Xy.map(lambda x: (x[1], predict(ws, x[0], b)))
    correct = y_and_y_hat.map(lambda x: 1 if x[0]==x[1] else 0).reduce(lambda x, y: x+y)
    return correct/total

In [19]:
def predict(w, X, b):
    tot = 0
    for xi, wi in zip(X, w.value):
        tot += xi * wi
    tot += b
    val = sigmoid(tot)
    if (val>=0.5):
        return 1.0
    return 0.0

## Testing

In [20]:
RDD_Xy = readFile("../data/botnet_tot_syn_l.csv")
RDD_Xy.count()

10011

In [21]:
RDD_Xy_normalized = normalize(RDD_Xy)

In [22]:
RDD_Xy_presplit = RDD_Xy_normalized.map(lambda x: (x[0], x[1], 1 if random.random() < 0.75 else 0))
train_data, test_data = trainTestSplit(RDD_Xy_presplit)
print("train_data")
print(train_data.count())
print("test_data")
print(test_data.count())

train_data


7523
test_data


2518


In [ ]:
w_final, b = train(train_data, 5, 0.1, 0.01)
b

Cost for it 0: 0.8263480397941587


Accuracy for it 0:0.3071608874717683


Cost for it 1: 0.850277644442121


Accuracy for it 1:0.30947905477980664


Cost for it 2: 0.846372993257511


Accuracy for it 2:0.310317248733671


Cost for it 3: 0.8678033759677501


Accuracy for it 3:0.3033453285352526


Cost for it 4: 0.8811189188073439


In [ ]:
acc = accuracy(w_final, b, test_data)
acc

In [ ]:
acc_train = accuracy(w_final, b, train_data)
acc_train